In [1]:
is_stack = [True, False][0]
debug = False
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error, roc_auc_score


#========================================================================
# Args
out_part = ['', 'part', 'all'][0]
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-24 11:08:01,702 utils 400 [INFO]    [logger_func] start 


In [2]:
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)

In [7]:
pred_col = 'prediction'
base = utils.read_df_pkl('../input/base_type_group.gz')[[key, 'first_active_month', target, 'clf_pred', 'no_out_flg', 'type']].set_index(key)
df_pred = pd.read_csv('../ensemble/good_submit_ensemble/tmp/0222_2031_alijs_stack_submit_pp_02222031_CV3.6059_LB3.6590.csv').set_index(key)['prediction']
base[pred_col] = df_pred
base.head()
train = base[~base[target].isnull()]
train.reset_index(inplace=True)

In [10]:
id_list = utils.read_pkl_gzip('../input/0217_merchant_one_id_list.gz')
tmp = train.loc[train[key].isin(id_list), [key, target]]
# id_list = tmp[tmp[target]<-30][key].values
train.loc[train[key].isin(id_list), :]

,card_id,first_active_month,target,clf_pred,no_out_flg,type,prediction
175,C_ID_7decdf7eec,2016-04-01,0.000000,0.002881,0.0,2,0.190887
818,C_ID_fba4a71df1,2015-11-01,0.135437,0.002387,0.0,2,0.161726
1100,C_ID_dc8b95d37f,2016-02-01,0.000000,0.029440,0.0,0,-0.474119
2061,C_ID_716e065493,2017-10-01,-0.487497,0.002798,0.0,1,0.194753
3442,C_ID_460aa35ea2,2015-12-01,0.000000,0.025907,0.0,0,-0.930114
3485,C_ID_5da6294302,2016-06-01,0.000000,0.002430,0.0,2,0.134784
4061,C_ID_9000088bfa,2016-03-01,0.149481,0.000605,1.0,10,0.179676
4089,C_ID_c94316139b,2016-09-01,0.000000,0.048857,0.0,0,-0.867598
4148,C_ID_1a4fbf387c,2017-11-01,0.000000,0.000441,1.0,10,0.240622
4712,C_ID_909bc065d5,2017-09-01,0.000000,0.003234,0.0,2,0.204104


In [35]:
df_f = df_hist.groupby(key)['purchase_amount_new'].first()
id_first1_list = list(df_f[df_f==1].index)
tmp = train.loc[train[key].isin(id_first1_list), :][[target, 'prediction', 'clf_pred']].sort_values(by=target, ascending=False)
tmp[target] = tmp[target].map(lambda x: np.round(x, 1))
tmp.groupby(target)[[pred_col, 'clf_pred']].mean()

,prediction,clf_pred
target,,
-33.2,-4.539467,0.121406
-15.4,-4.273871,0.089998
-9.6,-2.314139,0.024418
-9.0,-1.744587,0.001572
-6.9,-1.149828,0.002087
-6.8,-0.165874,0.000302
-6.4,-1.013590,0.004449
-6.3,-3.219364,0.020196
-6.2,-1.322140,0.002103


In [37]:
pd.set_option('max_row', 500)
# tmp = df_hist.loc[df_hist[key].isin(id_list), :]
tmp = df_hist.loc[df_hist[key].isin(id_first1_list), :]

train['raw_target'] = 2**train[target]
base_col = [col for col in train.columns if col.count('base_')]
# tmp[tmp[target]>3]
tmp.set_index([key, 'purchase_date'], inplace=True)
tmp = tmp.join(train.set_index([key])[[target, 'raw_target', 'clf_pred',pred_col]])
tmp.sort_values(by=[target], ascending=True, inplace=True)
tmp.sort_values(by=[key, 'purchase_date'], ascending=True, inplace=True)

In [42]:
viz_cols = ['authorized_flag', 'installments', 'merchant_id', 'month_lag', 'purchase_amount_new', target, 'raw_target', 'clf_pred', pred_col]
tmp = tmp[~tmp[target].isnull()][viz_cols]
tmp['diff'] = np.abs(tmp[target] - tmp[pred_col])
# tmp[tmp['diff']>10]
tmp[tmp[target]<-30]

authorized_flag  installments  \
card_id         purchase_date                                       
C_ID_2bb4f7bf28 2017-01-12 15:18:00               Y             0   
                2017-01-12 15:21:25               Y             0   
                2017-01-18 15:45:59               Y             0   
                2017-01-24 06:05:17               N             0   
                2017-01-24 06:06:43               Y             0   
                2017-01-24 15:03:51               N             0   
                2017-01-24 15:04:20               N             0   
                2017-02-06 16:32:30               Y             0   
                2017-02-22 08:28:38               Y             0   
                2017-02-24 08:46:43               Y             0   
                2017-02-26 09:09:09               Y             0   
                2017-02-28 09:07:51               Y             0   
                2017-03-02 11:35:30               Y             0   
                2017-03-02 21:13:44               N             0   
                2017-03-02 21:15:03               Y             0   
                2017-03-03 18:33:27               Y             0   
                2017-03-04 20:16:43               Y             0   
                2017-03-05 19:47:48               Y             0   
                2017-03-06 18:15:22               Y             0   
                2017-03-08 11:59:55               N             0   
                2017-03-12 16:50:13               N             0   
                2017-03-12 16:50:37               N             0   
                2017-03-13 20:54:41               Y             0   
                2017-03-15 10:13:35               Y             0   
                2017-03-22 21:11:26               Y             0   
                2017-03-27 21:15:31               Y             0   
                2017-05-03 18:50:08               Y             0   
                2017-05-20 10:17:42               Y             0   
                2017-05-20 15:40:13               Y             0   
                2017-05-21 10:19:20               Y             0   
                2017-05-21 10:21:26               Y             0   
                2017-05-21 10:28:28               Y             0   
                2017-05-21 14:06:31               N             0   
                2017-05-21 14:06:54               Y             0   
                2017-05-28 11:17:40               Y             0   
                2017-06-03 17:59:09               N             0   
                2017-06-03 17:59:36               N             0   
                2017-06-03 18:28:26               Y             0   
                2017-06-16 19:43:40               Y             0   
                2017-06-18 08:45:56               Y             0   
                2017-06-21 14:09:52               Y             0   
                2017-06-23 07:15:15               Y             0   
                2017-08-04 15:15:30               Y             0   
                2017-08-24 18:24:55               N             1   
                2017-08-24 18:25:36               N             1   
                2017-11-04 00:00:00               Y             0   
C_ID_2dd54c3384 2017-01-04 15:42:55               Y             1   
                2017-01-04 17:05:43               Y             1   
                2017-01-04 17:08:16               Y             1   
                2017-01-04 17:34:32               Y             1   
                2017-01-05 01:01:44               Y             1   
                2017-01-05 01:16:24               Y             1   
                2017-01-05 14:49:40               Y             1   
                2017-01-05 15:08:26               Y             1   
                2017-01-05 19:31:29               Y             1   
                2017-01-05 19:50:15               Y             1   
                2017-01-06 00:43:2

In [61]:
# df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7])
# df_last = df_hist.groupby(key)['yyyymm'].last()

# df_last = df_last.to_frame().reset_index()
# df_last = df_last.merge(df_hist, how='inner', on=[key, 'yyyymm'])
# print(df_last.shape)
# df_last['authorized_flag'] = df_last['authorized_flag'].map(lambda x: 1 if x=='Y' else 0)
# df_auth_last = df_last.groupby(key)['authorized_flag'].mean()
id_last_auth_list = df_auth_last[df_auth_last==0].index

In [77]:
use_cols = [key, target, pred_col, 'clf_pred']
# train_last_auth = train.loc[train[key].isin(id_last_auth_list), :]
train_last_auth[use_cols].sort_values(by=target).to_csv('../input/0224_elo_last_month_auth0_id_pred_LB3659.csv', index=False)